In [2]:
import os
import random
import numpy as np
import time
import datetime
import math

import torch
import torch.nn.functional as F 
import torchvision.datasets
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.transforms import v2
from torchvision.ops import nms
from torchmetrics.detection import MeanAveragePrecision as MAP
from PIL import ImageDraw
from tqdm import tqdm

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights,\
    fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights
from torchvision.models.detection.retinanet import retinanet_resnet50_fpn_v2, RetinaNet_ResNet50_FPN_V2_Weights
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights

from fedot.core.pipelines.pipeline_builder import PipelineBuilder

from fedcore.tools.ruler import PerformanceEvaluatorOD
from fedcore.architecture.dataset.object_detection_datasets import YOLODataset, COCODataset, UnlabeledDataset
from fedcore.architecture.comptutaional.devices import default_device
from fedcore.architecture.utils.loader import collate
from fedcore.data.data import CompressionInputData
from fedcore.inference.onnx import ONNXInferenceModel
from fedcore.neural_compressor.config import Torch2ONNXConfig
from fedcore.repository.constanst_repository import FEDOT_TASK
from fedcore.repository.initializer_industrial_models import FedcoreModels
from fedcore.repository.constanst_repository import CROSS_ENTROPY, MSE
from fedcore.architecture.visualisation.visualization import plot_train_test_loss_metric, apply_nms, get_image, filter_boxes
from fedcore.architecture.utils.loader import get_loader

c:\Users\Kaefsky\Python\Fedcore\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = default_device()
IMG_SIZE = 512
NMS_THRESH = 0.5
THRESH = 0.5

MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

INIT_LR = 4e-5

EPS = 10
BATCH_SIZE = 4

DATASET_NAME = 'dataset-5000'
OUTPUT_PATH = f'datasets/{DATASET_NAME}/output/images/'

In [4]:
transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    # v2.Normalize(mean=MEAN, std=STD),
    # v2.Resize((IMG_SIZE, IMG_SIZE))
])

tr_dataset = YOLODataset(path=f'datasets/{DATASET_NAME}', dataset_name=DATASET_NAME, train=True, log=True)
    
# train_dataset = COCODataset(images_path=COCO_PATH + "train2017/",
#                             json_path=COCO_PATH + "annotations/instances_train2017.json",
#                             transform=transform)

test_dataset = YOLODataset(path=f'datasets/{DATASET_NAME}', dataset_name=DATASET_NAME, train=False)
# val_dataset = COCODataset(images_path=COCO_PATH + "val2017/",
#                             json_path=COCO_PATH + "annotations/instances_val2017.json",
#                             transform=transform)
val_dataset = UnlabeledDataset(images_path=f'datasets/{DATASET_NAME}/val/images/')


tr_loader = get_loader(tr_dataset, batch_size=BATCH_SIZE, train=True)
test_loader = get_loader(test_dataset)
val_loader = get_loader(val_dataset)

# More accurate, very slow to train
# model = fasterrcnn_resnet50_fpn_v2()

# Less accurate, but faster to train
model = fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)

# test
# model = ssdlite320_mobilenet_v3_large(weights=SSDLite320_MobileNet_V3_Large_Weights.DEFAULT)
# model = retinanet_resnet50_fpn_v2()

num_classes = len(tr_dataset.classes)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes).to(device)
model.to(device)

opt = optim.SGD(model.parameters(), lr=INIT_LR, momentum=0.9, weight_decay=INIT_LR/2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='max', patience=3, verbose=True)

Data Path:  c:\Users\Kaefsky\Python\Fedcore\FedCore\datasets\dataset-5000


In [5]:
tr_loss = np.zeros(EPS)
test_loss = np.zeros(EPS)
tr_map = np.zeros(EPS)
test_map = np.zeros(EPS)
tr_time = np.zeros(EPS)

for ep in range(EPS):       
    tStart = time.time()
    
    # Train the model
    model.train()
    loss_arr = np.zeros(len(tr_loader))
    desc='Training'
    for i, (images, targets) in enumerate(tqdm(tr_loader, desc=desc)):
        # forward
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        loss_arr[i] = loss
        # backward + optimize
        opt.zero_grad()
        loss.backward()
        opt.step()           
    tr_loss[ep] = loss_arr.mean()
    
    # Calculate train mAP
    model.eval()
    evaluator = PerformanceEvaluatorOD(model, tr_loader, batch_size=BATCH_SIZE)
    target_metric = evaluator.measure_target_metric()
    tr_map[ep] = float(target_metric["map"])
            
    # Evaluate the model
    model.train()
    loss_arr = np.zeros(len(test_loader)) 
    desc='Evaluating'
    for i, (images, targets) in enumerate(tqdm(test_loader, desc=desc)):
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        loss_arr[i] = loss
    test_loss[ep] = loss_arr.mean()
    
    # Calculate test mAP
    model.eval()
    evaluator = PerformanceEvaluatorOD(model, test_loader, batch_size=1)
    target_metric = evaluator.measure_target_metric()
    test_map[ep] = float(target_metric["map"])
    
    # Optimize learning rate
    scheduler.step(test_map[ep])
    
    tEnd = time.time()
    tr_time[ep] = float(tEnd - tStart)
    
    # Print metrics
    p = int(math.log(ep + 1, 10))
    print('-' * (40 + p))
    print('| %d | TRAIN | Loss: %.3f | mAP: %.3f |' %
            (ep + 1, tr_loss[ep], tr_map[ep]))
    print('| %d | TEST  | Loss: %.3f | mAP: %.3f |' %
            (ep + 1, test_loss[ep], test_map[ep]))
    print('-' * (13 + p), 
            'Time: %.2f' % tr_time[ep], 
            '-' * 14)
    
    # Saving best model
    if test_map[ep].max():
        best_model = model
    
    # Most crucial step
    if device == 'cuda':
        torch.cuda.empty_cache()
    
    # Early stop
    if ep > 4 and test_map[ep] <= test_map[ep - 4]:
        tr_loss = tr_loss[:ep + 1]
        test_loss = test_loss[:ep + 1]
        tr_map = tr_map[:ep + 1]
        test_map = test_map[:ep + 1]
        train_time = tr_time[:ep + 1]
        print('Early stopping')
        break

# Final evaluating
model = best_model
evaluator = PerformanceEvaluatorOD(model, test_loader, batch_size=1)
performance = evaluator.eval()
print('Before quantization')
print(performance)

Training:   1%|          | 12/1051 [00:03<04:52,  3.55it/s]

KeyboardInterrupt



In [ ]:
plot_train_test_loss_metric(tr_loss, test_loss, tr_map, test_map)

In [ ]:
now = str(datetime.datetime.now())[2:-16]
torch.save(model, f'{model._get_name()}_{DATASET_NAME}_{now}.pt')

In [4]:
model = torch.load('FasterRCNN_african-wildlife_24-07-07.pt')

In [5]:
model.cpu()
id = random.randint(0, len(val_dataset) - 1) # random or int
test_data = test_loader.dataset[id]
img, target = test_data
input = torch.unsqueeze(img, dim=0)
pred = model(input)
pred = apply_nms(pred[0], NMS_THRESH)
pred = filter_boxes(pred, THRESH)

# Show inference image
transform = v2.ToPILImage()
img = transform(img)
inference_img = get_image(img, pred, train_dataset.classes, target)
inference_img.show()

In [6]:
# Predicting all inference images
for data in val_loader:
    image = data[0][0].cpu()
    name = data[1][0]['name']
    input = torch.unsqueeze(image, dim=0)
    pred = model(input)
    pred = apply_nms(pred[0], NMS_THRESH)
    pred = filter_boxes(pred, THRESH)
    transform = v2.ToPILImage()
    img = transform(image)
    inference_img = get_image(img, pred, train_dataset.classes)
    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)
    inference_img.save(OUTPUT_PATH + name)

In [ ]:
model = model.cpu()
repo = FedcoreModels().setup_repository()
compression_pipeline = PipelineBuilder().add_node('post_training_quant').build()

input_data = CompressionInputData(features=np.zeros((2, 2)),
                                    idx=None,
                                    calib_dataloader=val_loader,
                                    task=FEDOT_TASK['regression'],
                                    data_type=None,
                                    target=model
)

input_data.supplementary_data.is_auto_preprocessed = True
compression_pipeline.fit(input_data)
quant_model = compression_pipeline.predict(input_data).predict

In [ ]:
int8_onnx_config = Torch2ONNXConfig(
    dtype="int8",
    opset_version=18,
    quant_format="QDQ",  # or "QLinear"
    example_inputs=torch.unsqueeze(train_dataset[0][0], dim=0),
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
            'input' : {0 : 'batch_size'},
            'output' : {0 : 'batch_size'}
        }
)

quant_model.export("int8-model.onnx", int8_onnx_config)
onnx_model = ONNXInferenceModel("int8-model.onnx")

In [ ]:
evaluator = PerformanceEvaluatorOD(model, test_loader, batch_size=1)
performance = evaluator.eval()
print('after quantization')
print(performance)